In [ ]:
!pip install accelerate

In [ ]:
import transformers
import accelerate

# Load the pre-trained model
model = transformers.AutoModel.from_pretrained("facebook/bart-base")

# Create an Accelerate Accelerator
accelerator = accelerate.Accelerator()

# Move the model to the GPU
model.to(accelerator.device)

In [3]:
import csv

def load_dataset(csv_file):
    """Loads a code migration dataset from a CSV file.

    Args:
        csv_file: The path to the CSV file.

    Returns:
        A list of tuples, where each tuple contains the input and output code snippets.
    """

    dataset = []
    with open(csv_file, "r") as f:
        reader = csv.reader(f)

        for row in reader:
            input_code = row[0]
            output_code = row[1]

            dataset.append((input_code, output_code))

    return dataset

# Load the modified dataset
dataset = load_dataset("Data\code_to_code_geekforgeek.csv")
print(type(dataset))

<class 'list'>


In [5]:
print(len(dataset))

7121


In [6]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

def create_train_dataloader(dataset, batch_size):
    """Creates a training dataloader for a code migration dataset.

    Args:
        dataset: A list of tuples, where each tuple contains the input and output code snippets.
        batch_size: The batch size.

    Returns:
        A DataLoader object.
    """

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader

# Create the training dataloader
train_dataloader = create_train_dataloader(dataset, batch_size=16)


In [7]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

c:\Users\divya\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
from transformers import get_linear_schedule_with_warmup

num_training_steps = len(train_dataloader) * 2000

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)


In [21]:
for i in train_dataloader:
    print(i)
    break

[('Translate Java to C++: \nimport java.util. *;\nimport java.util.stream.*;\nimport java.lang.*;\nimport javafx.util.Pair;\npublic class C_PROGRAM_CONCATENATE_STRING_GIVEN_NUMBER_TIMES{\nstatic String f_gold ( String s , int n ) {\n  String s1 = s ;\n  for ( int i = 1 ;\n  i < n ;\n  i ++ ) s += s1 ;\n  return s ;\n}\n\n\n', 'Translate Python to C++: \nif __name__ == \'__main__\':\n    param = [\n    (2009.019461888707,),\n    (-1480.5131394215787,),\n    (357.7870347569567,),\n    (-8040.293697508038,),\n    (3821.882657293133,),\n    (-6840.635072240347,),\n    (1623.036598830132,),\n    (-9714.00706195298,),\n    (3916.454769669618,),\n    (-669.068424712943,)\n        ]\n    n_success = 0\n    for i, parameters_set in enumerate(param):\n        if abs(1 - (0.0000001 + abs(f_gold(*parameters_set))) / (abs(f_filled(*parameters_set)) + 0.0000001)) < 0.001:\n            n_success+=1\n    print("#Results: %i, %i" % (n_success, len(param)))', 'Translate C++ to Python: \nusing namespace 

In [26]:
import torch

In [27]:
# Start training
for epoch in range(2000):
    for batch in train_dataloader:
        input_text = ""
        for snippet_tuple in batch:
            input_text += snippet_tuple[0] + "\n" + snippet_tuple[1] + "\n"
        # Convert the list of code snippets to tensors
        batch = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024)
        # Move the batch to the GPU
        batch = batch.to(accelerator.device)
        print("To forward pass")
        # Forward pass
        outputs = model(**batch)

        # Loss
        print("To loss")
        loss = torch.mean(outputs[0])  # Compute the average loss across the batch.

        # Backward pass
        print("To backward pass")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the learning rate
        print("Updating learning rate")
        scheduler.step()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-bart-base-code-migration")

To forward pass
To loss
To backward pass
Updating learning rate
To forward pass
To loss
To backward pass
Updating learning rate
To forward pass
To loss
To backward pass


KeyboardInterrupt: 